<a href="https://colab.research.google.com/github/noora-ekramy/Sports_Classification_NNProject/blob/main/sports_classification_nnproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! unzip "/content/drive/MyDrive/NN Project Dataset/data.zip"

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

trainDataGenerator = ImageDataGenerator(rescale=1./255, validation_split=0.2 )


In [ ]:
classes=[
    'Basketball',
    'Football',
    'Rowing',
    'Swimming',
    'Tennis',
    'Yoga',
]

In [ ]:
train_d = trainDataGenerator.flow_from_directory(
     "/content/kaggle/working/images",
      classes=classes,
      color_mode="rgb",
      batch_size=32,
      target_size=(224,224),
      class_mode="categorical",
      subset="training")
valid_d = trainDataGenerator.flow_from_directory(
     "/content/kaggle/working/images",
      classes=classes,
      color_mode="rgb",
      batch_size=32,
      target_size=(224,224),
      class_mode="categorical",
      subset="validation")

Found 22717 images belonging to 6 classes.
Found 5677 images belonging to 6 classes.


In [ ]:
image_size = 224

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import concatenate
from keras.layers import Conv2D , MaxPool2D , Input , GlobalAveragePooling2D ,AveragePooling2D, Dense , Dropout ,Activation, Flatten , BatchNormalization


In [ ]:
def InceptionV3():
    input_layer = Input(shape=(224 , 224 , 3))

    x = StemBlock(input_layer)

    x = InceptionBlock_A(prev_layer = x ,nbr_kernels = 32)
    x = InceptionBlock_A(prev_layer = x ,nbr_kernels = 64)
    x = InceptionBlock_A(prev_layer = x ,nbr_kernels = 64)

    x = ReductionBlock_A(prev_layer = x )

    x = InceptionBlock_B(prev_layer = x  , nbr_kernels = 128)
    x = InceptionBlock_B(prev_layer = x , nbr_kernels = 160)
    x = InceptionBlock_B(prev_layer = x , nbr_kernels = 160)
    x = InceptionBlock_B(prev_layer = x , nbr_kernels = 192)

    Aux = auxiliary_classifier(prev_Layer = x)

    x = ReductionBlock_B(prev_layer = x)

    x = InceptionBlock_C(prev_layer = x)
    x = InceptionBlock_C(prev_layer = x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(units=2048, activation='relu') (x)
    x = Dropout(rate = 0.2) (x)
    x = Dense(units=6, activation='softmax') (x)

    model = Model(inputs = input_layer , outputs = [x,Aux] , name = 'Inception-V3')

    return model


def conv_with_Batch_Normalisation(prev_layer , nbr_kernels , filter_Size , strides =(1,1) , padding = 'same'):
    x = Conv2D(filters=nbr_kernels, kernel_size = filter_Size, strides=strides , padding=padding)(prev_layer)
    x = BatchNormalization(axis=3)(x)
    x = Activation(activation='relu')(x)
    return x


def StemBlock(prev_layer):
    x = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 32, filter_Size=(3,3) , strides=(2,2))
    x = conv_with_Batch_Normalisation(x, nbr_kernels = 32, filter_Size=(3,3))
    x = conv_with_Batch_Normalisation(x, nbr_kernels = 64, filter_Size=(3,3))
    x = MaxPool2D(pool_size=(3,3) , strides=(2,2)) (x)
    x = conv_with_Batch_Normalisation(x, nbr_kernels = 80, filter_Size=(1,1))
    x = conv_with_Batch_Normalisation(x, nbr_kernels = 192, filter_Size=(3,3))
    x = MaxPool2D(pool_size=(3,3) , strides=(2,2)) (x)
    return x


def InceptionBlock_A(prev_layer  , nbr_kernels):

    branch1 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 64, filter_Size = (1,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels=96, filter_Size=(3,3))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels=96, filter_Size=(3,3))

    branch2 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels=48, filter_Size=(1,1))
    branch2 = conv_with_Batch_Normalisation(branch2, nbr_kernels=64, filter_Size=(3,3)) # may be 3*3

    branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding='same') (prev_layer)
    branch3 = conv_with_Batch_Normalisation(branch3, nbr_kernels = nbr_kernels, filter_Size = (1,1))

    branch4 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels=64, filter_Size=(1,1))

    output = concatenate([branch1 , branch2 , branch3 , branch4], axis=3)

    return output


def InceptionBlock_B(prev_layer , nbr_kernels):

    branch1 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = nbr_kernels, filter_Size = (1,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = nbr_kernels, filter_Size = (7,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = nbr_kernels, filter_Size = (1,7))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = nbr_kernels, filter_Size = (7,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 192, filter_Size = (1,7))

    branch2 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = nbr_kernels, filter_Size = (1,1))
    branch2 = conv_with_Batch_Normalisation(branch2, nbr_kernels = nbr_kernels, filter_Size = (1,7))
    branch2 = conv_with_Batch_Normalisation(branch2, nbr_kernels = 192, filter_Size = (7,1))

    branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding ='same') (prev_layer)
    branch3 = conv_with_Batch_Normalisation(branch3, nbr_kernels = 192, filter_Size = (1,1))

    branch4 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 192, filter_Size = (1,1))

    output = concatenate([branch1 , branch2 , branch3 , branch4], axis = 3)

    return output


def InceptionBlock_C(prev_layer):

    branch1 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 448, filter_Size = (1,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 384, filter_Size = (3,3))
    branch1_1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 384, filter_Size = (1,3))
    branch1_2 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 384, filter_Size = (3,1))
    branch1 = concatenate([branch1_1 , branch1_2], axis = 3)

    branch2 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 384, filter_Size = (1,1))
    branch2_1 = conv_with_Batch_Normalisation(branch2, nbr_kernels = 384, filter_Size = (1,3))
    branch2_2 = conv_with_Batch_Normalisation(branch2, nbr_kernels = 384, filter_Size = (3,1))
    branch2 = concatenate([branch2_1 , branch2_2], axis = 3)

    branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding='same')(prev_layer)
    branch3 = conv_with_Batch_Normalisation(branch3, nbr_kernels = 192, filter_Size = (1,1))

    branch4 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 320, filter_Size = (1,1))

    output = concatenate([branch1 , branch2 , branch3 , branch4], axis = 3)

    return output


def ReductionBlock_A(prev_layer):

    branch1 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 64, filter_Size = (1,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 96, filter_Size = (3,3))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 96, filter_Size = (3,3) , strides=(2,2) ) #, padding='valid'

    branch2 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 384, filter_Size=(3,3) , strides=(2,2) )

    branch3 = MaxPool2D(pool_size=(3,3) , strides=(2,2) , padding='same')(prev_layer)

    output = concatenate([branch1 , branch2 , branch3], axis = 3)

    return output


def ReductionBlock_B(prev_layer):

    branch1 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 192, filter_Size = (1,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 192, filter_Size = (1,7))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 192, filter_Size = (7,1))
    branch1 = conv_with_Batch_Normalisation(branch1, nbr_kernels = 192, filter_Size = (3,3) , strides=(2,2) , padding = 'valid')

    branch2 = conv_with_Batch_Normalisation(prev_layer, nbr_kernels = 192, filter_Size = (1,1) )
    branch2 = conv_with_Batch_Normalisation(branch2, nbr_kernels = 320, filter_Size = (3,3) , strides=(2,2) , padding='valid' )

    branch3 = MaxPool2D(pool_size=(3,3) , strides=(2,2) )(prev_layer)

    output = concatenate([branch1 , branch2 , branch3], axis = 3)

    return output


def auxiliary_classifier(prev_Layer):
    x = AveragePooling2D(pool_size=(5,5) , strides=(3,3)) (prev_Layer)
    x = conv_with_Batch_Normalisation(x, nbr_kernels = 128, filter_Size = (1,1))
    x = Flatten()(x)
    x = Dense(units = 768, activation='relu') (x)
    x = Dropout(rate = 0.2) (x)
    x = Dense(units = 6, activation='softmax') (x)
    return x
x=InceptionV3()

x.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

history = x.fit(train_d,validation_data=valid_d,epochs=100)


710/710 [==============================] - 248s 349ms/step - loss: 0.0677 - dense_3_loss: 0.0340 - dense_1_loss: 0.0337 - dense_3_acc: 0.9890 - dense_1_acc: 0.9895 - val_loss: 0.1783 - val_dense_3_loss: 0.0899 - val_dense_1_loss: 0.0884 - val_dense_3_acc: 0.9711 - val_dense_1_acc: 0.9748
Epoch 31/100
710/710 [==============================] - 250s 353ms/step - loss: 0.0577 - dense_3_loss: 0.0305 - dense_1_loss: 0.0273 - dense_3_acc: 0.9904 - dense_1_acc: 0.9910 - val_loss: 0.3446 - val_dense_3_loss: 0.1770 - val_dense_1_loss: 0.1676 - val_dense_3_acc: 0.9524 - val_dense_1_acc: 0.9563
Epoch 32/100
710/710 [==============================] - 249s 350ms/step - loss: 0.0620 - dense_3_loss: 0.0331 - dense_1_loss: 0.0289 - dense_3_acc: 0.9897 - dense_1_acc: 0.9912 - val_loss: 0.5876 - val_dense_3_loss: 0.3032 - val_dense_1_loss: 0.2844 - val_dense_3_acc: 0.9320 - val_dense_1_acc: 0.9443
Epoch 33/100
710/710 [==============================] - 247s 348ms/step - loss: 0.0645 - dense_3_loss: 0.03

In [ ]:
x.save("InceptionV3_Model.h5")